<a href="https://colab.research.google.com/github/sam19980822/3rd-ML100Days/blob/master/D85_%E4%BD%BF%E7%94%A8_callbacks_%E5%87%BD%E6%95%B8%E5%81%9A_earlystop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import keras
import itertools
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization,Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [0]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [0]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [0]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)





Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               

## 什麼都不動

In [0]:
earlystop_1 = EarlyStopping(monitor="val_loss", 
                          patience=5, 
                          verbose=1
                          )

In [9]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[earlystop_1]
         )

# Collect results
train_loss1 = model.history.history["loss"]
valid_loss1 = model.history.history["val_loss"]
train_acc1 = model.history.history["acc"]
valid_acc1 = model.history.history["val_acc"]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples
Epoch 1/50





50000/50000 [==============================] - 11s 222us/step - loss: 2.1927 - acc: 0.2736 - val_loss: 2.1033 - val_acc: 0.3256
Epoch 2/50
50000/50000 [==============================] - 1s 24us/step - loss: 1.7380 - acc: 0.3910 - val_loss: 1.7674 - val_acc: 0.3933
Epoch 3/50
50000/50000 [==============================] - 1s 24us/step - loss: 1.6145 - acc: 0.4337 - val_loss: 1.6698 - val_acc: 0.4162
Epoch 4/50
50000/50000 [==============================] - 1s 24us/step - loss: 1.5450 - acc: 0.4604 - val_loss: 1.6316 - val_acc: 0.4271
Epoch 5/50
50000/50000 [==============================] - 1s 24us/step - loss: 1.4875 - acc: 0.4799 - val_loss: 1.5852 - val_acc: 0.4387
Epoch 6/50
50000/50000 [==============================] - 1s 24us/step - loss: 1.4423 - acc: 0.4940 - val_loss: 1.5667 - val_acc: 0.4506
Epoch 7/50
50000/50000 [

## 1.改變 monitor "Validation Accuracy" 並比較結果


In [0]:
earlystop_2 = EarlyStopping(monitor="val_acc", 
                          patience=5, 
                          verbose=1
                          )

In [11]:

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[earlystop_2]
         )

# Collect results
train_loss2 = model.history.history["loss"]
valid_loss2 = model.history.history["val_loss"]
train_acc2 = model.history.history["acc"]
valid_acc2 = model.history.history["val_acc"]

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 1s 25us/step - loss: 0.8964 - acc: 0.6999 - val_loss: 1.4773 - val_acc: 0.5025
Epoch 2/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.8749 - acc: 0.7097 - val_loss: 1.4815 - val_acc: 0.5025
Epoch 3/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.8557 - acc: 0.7176 - val_loss: 1.4845 - val_acc: 0.5037
Epoch 4/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.8352 - acc: 0.7268 - val_loss: 1.4977 - val_acc: 0.4974
Epoch 5/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.8160 - acc: 0.7328 - val_loss: 1.4958 - val_acc: 0.4952
Epoch 6/50
50000/50000 [==============================] - 1s 25us/step - loss: 0.7979 - acc: 0.7403 - val_loss: 1.4981 - val_acc: 0.4991
Epoch 7/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.7770 - acc: 0.7467 - val_loss: 1.5039 - val_acc

## 2.調整 earlystop 的等待次數至 10, 25 並比較結果

In [0]:
earlystop_3 = EarlyStopping(monitor="val_loss", 
                          patience=10, 
                          verbose=1
                          )

In [13]:

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[earlystop_3]
         )

# Collect results
train_loss3 = model.history.history["loss"]
valid_loss3 = model.history.history["val_loss"]
train_acc3 = model.history.history["acc"]
valid_acc3 = model.history.history["val_acc"]

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.7393 - acc: 0.7625 - val_loss: 1.5570 - val_acc: 0.4881
Epoch 2/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.7216 - acc: 0.7700 - val_loss: 1.5253 - val_acc: 0.4982
Epoch 3/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.7019 - acc: 0.7762 - val_loss: 1.5599 - val_acc: 0.4969
Epoch 4/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.6840 - acc: 0.7846 - val_loss: 1.5437 - val_acc: 0.4917
Epoch 5/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.6638 - acc: 0.7933 - val_loss: 1.5700 - val_acc: 0.4982
Epoch 6/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.6478 - acc: 0.7978 - val_loss: 1.5798 - val_acc: 0.4945
Epoch 7/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.6296 - acc: 0.8065 - val_loss: 1.5837 - val_acc

In [0]:
earlystop_4 = EarlyStopping(monitor="val_loss", 
                          patience=25, 
                          verbose=1
                          )

In [15]:

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[earlystop_4]
         )

# Collect results
train_loss4 = model.history.history["loss"]
valid_loss4 = model.history.history["val_loss"]
train_acc4 = model.history.history["acc"]
valid_acc4 = model.history.history["val_acc"]

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.5259 - acc: 0.8484 - val_loss: 1.6852 - val_acc: 0.4881
Epoch 2/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.5095 - acc: 0.8554 - val_loss: 1.6604 - val_acc: 0.4915
Epoch 3/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.4932 - acc: 0.8617 - val_loss: 1.6662 - val_acc: 0.4946
Epoch 4/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.4769 - acc: 0.8687 - val_loss: 1.6914 - val_acc: 0.4917
Epoch 5/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.4613 - acc: 0.8741 - val_loss: 1.6859 - val_acc: 0.4955
Epoch 6/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.4462 - acc: 0.8798 - val_loss: 1.7153 - val_acc: 0.4920
Epoch 7/50
50000/50000 [==============================] - 1s 24us/step - loss: 0.4303 - acc: 0.8870 - val_loss: 1.7211 - val_acc

## 結論：
1.使用val_loss,patience=5 , 在第27epoch停住，train_acc=0.6938, val_loss = 1.4  
2.使用val_acc, patience=5 , 在第08epoch停住，train_acc=0.7551, val_loss = 1.5  
3.使用val_loss, patience=10 , 在第12epoch停住，train_acc=0.8411, val_loss = 1.6  
4.使用val_loss, patience=25 , 在第27epoch停住，train_acc=0.9695, val_loss = 2.1  

## 整體來說，patience不要訂太高，使用val_acc會最好